In [2]:
import os
os.chdir('..')

In [2]:
from src.api.workflows.FAQs.extract_FAQs import obtener_faqs, obtener_faqs_unir
faqs = obtener_faqs(); faqs_unir = obtener_faqs_unir()

In [3]:
# Leer el archivo y extraer los ejemplos  
def leer_ejemplos(archivo):  
    with open(archivo, 'r', encoding='utf-8') as file:  
        contenido = file.read()  
      
    ejemplos = contenido.split('EJEMPLO: ')[1:]  # Dividir por 'EJEMPLO: ' y omitir el primer elemento vacío  
    ejemplos = [ejemplo.strip() for ejemplo in ejemplos]  # Limpiar espacios en blanco al inicio y al final  
    return ejemplos  
  
# Usar la función  
preguntas_test = leer_ejemplos('docs\EJEMPLOS PREGUNTAS TÉCNICAS Y OTRAS FAQ.txt') 

In [4]:
from src.api.common.dependency_container import DependencyContainer
from IPython.display import HTML  
from tqdm import tqdm

output = []
DC = DependencyContainer(); DC.initialize()
for pregunta in tqdm(preguntas_test):
    request = {'Pregunta': pregunta, 'FAQs': faqs + faqs_unir}
    response = DependencyContainer.get_schema_generator_workflow().execute(request)
    # HTML(f"<h2>{response['FAQ correspondiente']}</h2>"+response['Contenido'])
    output.append(response['FAQ correspondiente'])

100%|██████████| 48/48 [04:39<00:00,  5.82s/it]


In [5]:
import pandas
df = pandas.DataFrame()
df['Pregunta'] = preguntas_test
df['gpt-3.5-turbo'] = output
df.to_excel('metricas_FAQs.xlsx')

In [30]:
import pandas as pd  
import plotly.express as px  
  
# Cargar datos  
df = pd.read_excel('docs/metricas_FAQS.xlsx', index_col=0).iloc[:, :3]  

# Calcular las frecuencias relativas en porcentaje  
counts = df['gpt-3.5-turbo'].value_counts().sort_index()  
percentages = (counts / counts.sum()) * 100  
# Crear un gráfico de barras con los porcentajes  
fig = px.bar(percentages,  
             title='Histograma de Métricas FAQs en Porcentaje',  
             labels={'index': 'Valores', 'value': 'Frecuencia (%)'},  
             template='plotly_white')  

# Mejorar visualmente el gráfico  
fig.update_layout(  
    showlegend=False,  # Ocultar la leyenda  
    xaxis_tickfont=dict(size=6),  # Reducir el tamaño de los ticks del eje X  
    xaxis_title="",  # Eliminar el título del eje X  
    yaxis=dict(  
        tickformat=".2f"  # Formato de dos decimales para los porcentajes  
    )  
)  
# Mostrar el gráfico  
fig.show()  

In [ ]:
import plotly.express as px  
import pandas as pd

df = pd.read_excel('docs/metricas_FAQS.xlsx', index_col=0).iloc[:,:3]

# Calcular las frecuencias relativas en porcentaje  
value_counts = df['metrica'].value_counts(normalize=True) * 100  # Esto convierte las cuentas en porcentajes  
value_counts = value_counts.sort_index()  # Asegurarse de que los índices estén ordenados para coincidir con las etiquetas personalizadas  
  
# Crear un gráfico de barras  
fig = px.bar(value_counts,  
             title='Histograma de Métricas FAQs en Porcentaje',  
             labels={'index': 'Valores', 'value': 'Frecuencia (%)'},  
             template='plotly_white')  
  
# Mejorar visualmente el gráfico  
fig.update_layout(  
    showlegend=False,  # Ocultar la leyenda  
    xaxis=dict(  
        tickmode='array',  
        tickvals=[0, 1, 2, 3],  
        ticktext=['Indeterminado', 'Malo', 'Regular', 'Bien']  
    ),  
    yaxis=dict(  
        tickformat=".0f"  # Formato de dos decimales para los porcentajes  
    )  
)  
fig.show()  